In [8]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("C:\\Users\\DELL\\Generative_AI\\Langchain\\apj_speech.pdf")
pages=loader.load_and_split()
pages

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'C:\\Users\\DELL\\Generative_AI\\Langchain\\apj_speech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st

In [9]:
from langchain.prompts import PromptTemplate
template=""" 
You are an expertise in summarizing the speech.Please provide it in concise and short manner of following speech,
speech:{text}
"""
prompt=PromptTemplate(input_variables=['text'],template=template)

In [11]:
from langchain.chains.summarize import load_summarize_chain
from langchain_groq import ChatGroq
import os

api_key=os.getenv("GROQAI_API_KEY")
llm=ChatGroq(temperature=0,api_key=api_key,model_name="Llama3-8b-8192")
chain=load_summarize_chain(llm,chain_type="stuff",prompt=prompt,verbose=True)
output=chain.run(pages)
print(output)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
You are an expertise in summarizing the speech.Please provide it in concise and short manner of following speech,
speech:A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state

In [15]:
# Map Reduce 
loader_1=PyPDFLoader(r"C:\\Users\\DELL\\Generative_AI\\Langchain\\NIPS-2017-attention-is-all-you-need-Paper.pdf")
docs=loader_1.load()
docs

[Document(metadata={'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'publisher': 'Curran Associates, Inc.', 'language': 'en-US', 'created': '2017', 'eventtype': 'Poster', 'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requiring significantly less timeto train. Our single model with 165 million parameters, achieves 27.5 BLEU onEnglish-to-German translation, improving over the existing best ensemble result by over 1 BLEU. On 

In [16]:

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
final_docs=text_splitter.split_documents(docs)
final_docs

[Document(metadata={'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'publisher': 'Curran Associates, Inc.', 'language': 'en-US', 'created': '2017', 'eventtype': 'Poster', 'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requiring significantly less timeto train. Our single model with 165 million parameters, achieves 27.5 BLEU onEnglish-to-German translation, improving over the existing best ensemble result by over 1 BLEU. On 

In [17]:
chunks_prompt=""" 
You are an expertise in summarizing the text. Pleas give it in short and concise manner
speech:{text}
Also translate the summarized text to {language}
"""
map_reduce_prompt=PromptTemplate(input_variables=['text','language'],template=chunks_prompt)
map_reduce_prompt

PromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, template=' \nYou are an expertise in summarizing the text. Pleas give it in short and concise manner\nspeech:{text}\nAlso translate the summarized text to {language}\n')

In [18]:
final_prompt=""" 
Provide a summarized text along with a apt heading by taking up the speech
Precise summarized text with good introduction and important points should be in bullet points
speech:{text}
and translate it to {language}
"""
final_prompt_template=PromptTemplate(input_variables=['text','language'],template=final_prompt)
final_prompt_template

PromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, template=' \nProvide a summarized text along with a apt heading by taking up the speech\nPrecise summarized text with good introduction and important points should be in bullet points\nspeech:{text}\nand translate it to {language}\n')

In [23]:
final_chain=load_summarize_chain(llm=llm,chain_type="map_reduce",verbose=True,map_prompt=map_reduce_prompt,
                                 combine_prompt=final_prompt_template)
output_1 = final_chain.run({"input_documents": final_docs, "language": "English"})



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
You are an expertise in summarizing the text. Pleas give it in short and concise manner
speech:Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗†
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser ∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrenc

Token indices sequence length is longer than the specified maximum sequence length for this model (8112 > 1024). Running this sequence through the model will result in indexing errors




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
Provide a summarized text along with a apt heading by taking up the speech
Precise summarized text with good introduction and important points should be in bullet points
speech:Here is a concise summary of the speech "Attention Is All You Need":

The authors propose a new neural network architecture, the Transformer, which uses attention mechanisms only, eliminating the need for recurrent or convolutional networks. This architecture outperforms existing models on two machine translation tasks, while being more parallelizable and requiring less computational resources.

And here is the translation of the summarized text to English:

"The authors propose a new neural network architecture, the Transformer, which uses attention mechanisms only, eliminating the need for recurrent or convolutional networks. This architecture outperforms existing models on two machine translation tasks,